In [7]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Embedding, Dropout

In [8]:
data_dir = "../Data2/"
data_file = "music_input.txt"
weights_dir = "../Data2/weights/"
json_file = "char_index.json"
batch_size = 16
seq_len = 64

# Model Building

In [9]:
def model_build(unique_chars) :
    
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1,1)))
              
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
              
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
              
    model.add(LSTM(256, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(Dense(unique_chars))
    model.add(Activation("Softmax"))
              
    return model
    

# Testing Model

In [10]:
def music_gen(epoch_no, init_index, len_seq) :
    
    with open(os.path.join(data_dir, json_file), mode = "r") as f :
        char_ind = json.load(f)
        
    ind_char = {i : ch for (ch,i) in char_ind.items()}
    unique_chars = len(ind_char)
    
    model = model_build(unique_chars)
    model.load_weights(weights_dir+"Weights_{}.h5".format(epoch_no))
    
    sequence_ind = [init_index]
    
    for i in range(len_seq) :
        batch = np.zeros((1,1))
        batch[0,0] = sequence_ind[-1]
        
        predicted_prob = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_prob)
        
        sequence_ind.append(sample[0])
        
    seq = "".join(ind_char[i] for i in sequence_ind)  
    
    c = 0
    for i in seq :
        c += 1
        if i == "\n" :
            break
    seq1 = seq[c:]
    
    c = 0
    for i in seq1 :
        c += 1
        if i == "\n" and seq1[c] == "\n" :
            break
    seq2 = seq1[c:]
    
    return seq2
          

In [11]:
epoch_no = int(input())
init_ind = int(input())
len_seq = int(input())
print(music_gen(epoch_no, init_ind, len_seq))

90
80
400

X: 24
T:Blackthery Qorkide
% Nottingham Music Database
S:Trad, arr Phil Rowe
M:6/8
K:C
E2F |"C"G3 "F7"A2F|"F"C3 "D7"ABc|
"G"B3 -"D7"B2c|"G"d2B GBd|"C"e3 "Em"ded|"Am"c3 "D7"A3|
"G"GAG "C"g2e|"G"dBG "D7"ABA|"G"G3 -G2:|
P:B
d|"D7"c2d e2f|"G"g2e "Em"dcB|"Am"A2A "D7"A2A|"G"B2G DBc|
"G"d2d "A7"ede|"D"f2d fgf|"D7"e2f def|"G"g3 g2f|
"Am"e2f "D7"e2f|
